In [1]:
using DataFrames;
using Distributions;

In [2]:
df = readtable("/Users/nwelch/prelim/data/infectionDataTest.csv");
dst = readtable("/Users/nwelch/prelim/data/infectedDistancesTest.csv");

# Open Issues: 
* I am currently using the shape and scale parameterization of the Gamma distribution. How can I confirm this is correct? I started by interpreting the mean in terms of the generative process. 
* Note that the last log sums in llRatio_tau have reversed signs than what is shown in the appendix. The Appendix appears to have the wrong sign. 

# Initial Parameter Values

In [313]:
#priors
mu_a=0.7; mu_b=0.004;
theta_a=0.8; theta_b=10;
sigma_a=0.5; sigma_b=100;

# initial/test values
u = mu_a*mu_b
th = theta_a*theta_b;
s = sigma_a*sigma_b;
t = [14., 10., 6.];
tl = maximum(t);

# Function Definitions

## log  Lambda

**Scrubbed and equivalent to R-unit test results.**

In [176]:
function getLogLambda(tau, theta, mu, sigma, dst, Tlast=30)
    iLessTlast = find(x-> x<=Tlast, tau)
    gaussian = Normal(mu, sigma)
    
    sumLog = 0
    for i = iLessTlast
        sumTauJLessTauI = 0
        jLessI = find(x-> x < tau[i], tau)

        for j = jLessI
            sumTauJLessTauI = sumTauJLessTauI + pdf(gaussian, dst[i,j])
        end
        sumLog = sumLog + log(mu + theta*sumTauJLessTauI)
    end
    sumLog
end

getLogLambda (generic function with 2 methods)

In [177]:
getLogLambda(t, th, u, s, dst, tl)

-10.623570479769231

## $\mu$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [251]:
muPrior = Gamma(mu_a, mu_b)

function llRatio_mu(mu, muStar, theta, sigma, tau, dst, Tlast=30, mu_a=0.7, mu_b=0.004)
    tauLessTIndex = find(x-> x<=Tlast, tau)
    tauGreaterTIndex = find(x-> x>Tlast, tau)
    
    dmu = mu - muStar
    
    dllTerm1 = sum(tau[tauLessTIndex]*dmu)
    dllTerm2 = length(tauGreaterTIndex) * Tlast * dmu
    
    dllTerm3a = getLogLambda(tau, theta, muStar, sigma, dst, Tlast)
    dllTerm3b = getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    dllTerm3 = dllTerm3a - dllTerm3b
    
    logPriorStar = log(pdf(muPrior, muStar))
    logPrior = log(pdf(muPrior, mu))
    
    dll = dllTerm1 + dllTerm2 + dllTerm3 + logPriorStar - logPrior
    dll
end

llRatio_mu (generic function with 4 methods)

In [254]:
llRatio_mu(u, 0.01, th, s, t, dst, tl)

-0.9685811338229993

## $\sigma$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [168]:
function getSumLessT_sigma(sigma, sigmaStar, mu, tau, theta, dst, Tlast=30.)
    tauLessTIndex = find(x -> x<=Tlast, tau) 
    gaussian = Normal(mu, sigma)
    gaussianStar = Normal(mu, sigmaStar)
    
    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            tmp = (tau[i] - tau[j])*(pdf(gaussian, dst[i,j]) -
                                   pdf(gaussianStar, dst[i,j]))
            sum_ij = sum_ij + tmp 
        end
        sumLessT = sumLessT + theta*sum_ij
    end
    sumLessT
end

getSumLessT_sigma (generic function with 2 methods)

In [171]:
getSumLessT_sigma(s, 1., u, t, th, dst, tl)

-37.07198157467801

In [186]:
function getSumGreaterT_sigma(sigma, sigmaStar, mu, tau, theta, dst, Tlast=30.)

    tauGreaterTIndex = find(x -> x>Tlast, tau)
    gaussian = Normal(mu, sigma)
    gaussianStar = Normal(mu, sigmaStar)
    
    sumGreaterT = 0
    for i=tauGreaterTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            sum_ij = sum_ij + 
                (Tlast - tau[j])*(pdf(gaussian, dst[i,j]) -
                                  pdf(gaussianStar, dst[i,j]))
        end
        sumGreaterT = sumGreaterT + theta*sum_ij
    end
    sumGreaterT
end

getSumGreaterT_sigma (generic function with 2 methods)

In [188]:
getSumGreaterT_sigma(s, 100., u, t, th, dst, 10.)

0.12761710295072787

In [189]:
sigmaPrior = Gamma(sigma_a, sigma_b)

function llRatio_sigma(sigma, sigmaStar, mu, tau, theta, dst, Tlast=30.,
        sigma_a=0.5, sigma_b=100.)
    
    dllTerm1 = getSumLessT_sigma(sigma, sigmaStar, mu, tau, theta, dst)

    dllTerm2 = getSumGreaterT_sigma(sigma, sigmaStar, mu, tau, theta, dst)
    
    dllTerm3 = getLogLambda(tau, theta, mu, sigmaStar, dst, Tlast) -
        getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    
    priorStar = pdf(sigmaPrior, sigmaStar)
    prior = pdf(sigmaPrior, sigma)
    
    dll = dllTerm1 + dllTerm2 + dllTerm3 + priorStar - prior
    
    dll
end

llRatio_sigma (generic function with 4 methods)

In [191]:
llRatio_sigma(s, 1., u, t, th, dst, tl)

-29.676456732636904

# $\theta$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [259]:
function getSumLessT_theta(theta, thetaStar, mu, tau, sigma, dst, Tlast=30.)
    gaussian = Normal(mu, sigma)
    
    tauLessTIndex = find(x -> x<=Tlast, tau)
    
    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j = tauJLessTauIIndex
            sum_ij = sum_ij + 
                (tau[i] - tau[j])*pdf(gaussian, dst[i,j])
        end
        sumLessT = sumLessT + (theta - thetaStar)*sum_ij
    end
    sumLessT
end

getSumLessT_theta (generic function with 2 methods)

In [269]:
getSumLessT_theta(th, 1., u, t, s, dst, tl)

0.893519762891437

In [261]:
function getSumGreaterT_theta(theta, thetaStar, mu, tau, sigma, dst, Tlast=30.)
    
    gaussian = Normal(mu, sigma)
    tauGreaterTIndex = find(x -> x>Tlast, tau)
    
    sumGreaterT = 0
    for i=tauGreaterTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            sum_ij = sum_ij + 
                (Tlast - tau[j])*pdf(gaussian, dst[i,j])
        end
        sumGreaterT = sumGreaterT + (theta - thetaStar)*sum_ij
    end
    sumGreaterT
end

getSumGreaterT_theta (generic function with 2 methods)

In [271]:
getSumGreaterT_theta(th, 1., u, t, s, dst, 10.)

0.2233632497737195

In [282]:
thetaPrior = Gamma(theta_a, theta_b)

function llRatio_theta(theta, thetaStar, mu, tau, sigma, dst, 
                         Tlast=30, theta_a=0.8, theta_b=10)
    
    dllTerm1 = getSumLessT_theta(theta, thetaStar, mu, tau, sigma, dst, Tlast)
    
    dllTerm2 = getSumGreaterT_theta(theta, thetaStar, mu, tau, sigma, dst, Tlast)
    
    dllTerm3 = getLogLambda(tau, thetaStar, mu, sigma, dst, Tlast) - 
                    getLogLambda(tau, theta, mu, sigma, dst, Tlast)
      
    logPriorStar = log(pdf(thetaPrior, thetaStar))
    logPrior = log(pdf(thetaPrior, theta))
    
    dll = dllTerm1 + dllTerm2 + dllTerm3 + logPriorStar - logPrior
    dll
end

llRatio_theta (generic function with 4 methods)

In [283]:
llRatio_theta(th, 1., u, t, s, dst, tl)

-1.7516207513257855

# $\tau$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [307]:
function getMinSum(iStar, tauiStar, mu, tau, theta, sigma, dst)
    gaussian = Normal(mu, sigma)
    ti = tau[iStar]
    minTau = min(tauiStar, ti)

    jLessMin = find(x -> x<minTau, tau)
    minSum = 0
    for j = jLessMin
        minSum = minSum + pdf(gaussian, dst[iStar,j])
    end
    minSum = (ti - tauiStar)*theta*minSum
    minSum
end

getMinSum (generic function with 1 method)

In [288]:
getMinSum(1, t[1]+0.5, u, t, th, s, dst)

-0.06382284020653121

In [289]:
function getMaxSum(iStar, tauiStar, mu, tau, theta, sigma, dst)
    gaussian = Normal(mu, sigma)
    ti = tau[iStar]
    maxTau = max(tauiStar, ti)
    
    jGreaterMax = find(x -> x>maxTau, tau)
    maxSum = 0
    for j = jGreaterMax
        maxSum = maxSum + pdf(gaussian, dst[iStar,j])
    end
    maxSum = (tauiStar - ti)*theta*maxSum
    maxSum
end

getMaxSum (generic function with 1 method)

In [293]:
getMaxSum(2, t[2]-0.5, u, t, th, s, dst)

-0.031913804524571286

In [294]:
function getSumIntervalBelowTauStar(iStar, tauiStar, mu, tau, theta, sigma, dst)
    
    gaussian = Normal(mu, sigma)
    ti = tau[iStar]
    
    jInBetween = find(x -> ((x>ti) & (x<tauiStar)), tau)
    sumBelow = 0
    for j = jInBetween
        sumBelow = sumBelow +
            (2*tau[j] - ti - tauiStar)*pdf(gaussian, dst[iStar,j])
    end
  sumBelow = theta*sumBelow
  sumBelow
end

getSumIntervalBelowTauStar (generic function with 1 method)

In [297]:
getSumIntervalBelowTauStar(2, 15., u, t, th, s, dst)

0.1914828271474277

In [298]:
function getSumIntervalAboveTauStar(iStar, tauiStar, mu, tau, theta, sigma, dst)
    gaussian = Normal(mu, sigma)
    ti = tau[iStar]
    
    jInBetween = find(x -> ((x>tauiStar) & (x<ti)), tau)
    sumAbove = 0
    for j = jInBetween
            sumAbove = sumAbove +
                (ti + tauiStar - 2*tau[j])*pdf(gaussian, dst[iStar,j])
    end
    sumAbove = theta*sumAbove
    sumAbove
end

getSumIntervalAboveTauStar (generic function with 1 method)

In [301]:
getSumIntervalAboveTauStar(1, 8., u, t, th, s, dst)

0.12765521809828514

In [324]:
function llRatio_tau(tau, tauiStar, iStar, mu, theta, sigma, dst, Tlast=30.)
    N = length(tau)
    
    dllTerm1 = mu*(tau[iStar] - tauiStar)
    dllTerm2 = getMinSum(iStar, tauiStar, mu, tau, theta, sigma, dst)
    dllTerm3 = getMaxSum(iStar, tauiStar, mu, tau, theta, sigma, dst)
    dllTerm4 = getSumIntervalBelowTauStar(iStar, tauiStar, mu, tau, 
                                          theta, sigma, dst)
    dllTerm5 = getSumIntervalAboveTauStar(iStar, tauiStar, mu, tau, 
                                          theta, sigma, dst)
    
    tauStar = copy(tau)
    tauStar[iStar] = tauiStar
    dllTerm6 = getLogLambda(tauStar, theta, mu, sigma, dst, Tlast)
    dllTerm7 = getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    
    dll = dllTerm1 + dllTerm2 + dllTerm3 + dllTerm4 + dllTerm5 +
        dllTerm6 - dllTerm7
    dll
end

llRatio_tau (generic function with 2 methods)

In [325]:
llRatio_tau(t, t[1]-0.5, 1, u, th, s, dst, tl)

0.06522284020653046